# Revised case normalization for Hirslanden Beau-Site 2017

Notes:

- There is no data for the hospital 'Hirslanden Beau Site' in 2017 in the DB.


In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(os.path.join(ROOT_DIR, 'raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx'),
                     'Hirslanden Beau Site', '2017', 'Änderungen Beau Site 2017')
print(file_info)


FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx', hospital_name_db='Hirslanden Beau Site', year='2017', sheet='Änderungen Beau Site 2017')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-11 07:11:32.856 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:35 - Read 47 cases for Hirslanden Beau Site 2017
2022-11-11 07:11:32.866 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:63 - TYPES:
datum                             string
tranche                           string
case_id                           object
patient_id                        object
gender                            string
age_years                          int64
bfs_code                          string
duration_of_stay                   int64
pflegetage neu                    string
pccl                               int64
pccl neu                          string
old_pd                            string
primary_diagnosis                 string
added_icds                        string
removed_icds                      string
added_chops                       string
removed_chops                     string
drg                         

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,41225861,D9798CB1880ECA77,M,63,5,41225861,C180,C180,M200,3,G10B,[C786],[],[],[]
1,41246909,1CF41AC1C2D10D62,M,83,3,41246909,J22,J22,M100,3,E75C,[D706],[],[],[]
2,41250682,89A046670053E7A4,W,76,17,41250682,K9180,K9180,M100,3,G70B,"[D630, D706]",[],[],[]
3,41256307,F14F340A6A8A29A5,W,77,5,41256307,N390,N390,M100,3,L63D,[K743],[],[],[]
4,41258314,35B85BE2EFA7811C,W,81,9,41258314,C56,C56,M100,3,N60B,[A099],[],[],[]
5,41272582,32E9366FA2B87CBA,M,55,4,41272582,K922,K922,M100,3,G48B,"[K500, K703, E871]",[K521],[],[]
6,41279857,150338288151836,M,64,9,41279857,I210,I210,M100,3,F41B,"[I1100, I313, I513]",[I1090],[],[]
7,41300053,EC6D4A9D2B502F76,M,66,6,41300053,C61,C61,M200,3,M01B,[C775],[],[],[]
8,41301906,EA46555953051562,W,81,5,41301906,K625,K625,M100,3,G48B,"[D500, I441]",[],[],[]
9,41269269,0BE197D58B3DE27B,W,72,15,41269269,I7029,I7029,M200,3,F33D,"[B956, M8627]",[],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases

ValueError: There is no data for the hospital 'Hirslanden Beau Site' in 2017

In [ ]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [ ]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

In [ ]:
revisions_update

In [ ]:
update_db(revisions_update, diagnoses_update, procedures_update)